<img src="../idp.jpg">

# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Aula 3 - Segmentação de Clientes (Clusterização)</p>

Neste projeto, estudaremos um caso prático utilizando técnicas de agrupamento não supervisionado considerando registros de clientes de um supermercado. A segmentação de clientes é a prática de separar os clientes em grupos que possuem características semelhantes em cada cluster. Iremos dividir os clientes em segmentos para otimizar a importância de cada cliente para o negócio. O resultado da clusterização também ajuda a empresa a atender às preocupações de diferentes tipos de clientes.

Link interessante sobre clusterização: conceitos básicos, principais algoritmos e aplicações.  
https://medium.com/turing-talks/clustering-conceitos-b%C3%A1sicos-principais-algoritmos-e-aplica%C3%A7%C3%A3o-ace572a062a9

**Sobre os dados**:

A Análise de Personalidade do Cliente é uma análise detalhada dos clientes ideais de uma empresa. Ajuda uma empresa a entender melhor seus clientes e facilita a modificação de produtos de acordo com as necessidades, comportamentos e preocupações específicas de diferentes tipos de clientes.

A análise da personalidade do cliente ajuda uma empresa a modificar seu produto com base em seus clientes-alvo de diferentes tipos de segmentos de clientes. Por exemplo, em vez de gastar dinheiro para comercializar um novo produto para cada cliente no banco de dados da empresa, uma empresa pode analisar qual segmento de cliente tem mais probabilidade de comprar o produto e, em seguida, comercializar o produto apenas nesse segmento específico.

Colunas do nosso conjunto de dados:
- ID: Identificador do cliente
- Education: escolaridade
- Marital_Status: estado civil
- Income: salário da família
- Kidhome: número de crianças
- Teenhome: número de adolescentes
- Dt_Customer: data do cadastro do cliente na empresa
- Recency: número de dias desde a última compra
- MntWines: valor gasto em vinho
- MntFruits: valor gasto em frutas
- MntMeatProducts: valor gasto em carne
- MntFishProducts: valor gasto em peixe
- MntSweetProducts: valor gasto em doces
- MntGoldProds: valor gasto em produtos gourmet
- NumDealsPurchases: número de compras feitas por promoções
- NumWebPurchases: número de compras feita pela web
- NumCatalogPurchases: número de compras feitas por catálogo
- NumStorePurchases: número de compras na loja
- NumWebVisitsMonth: número de visitas na web por mês
- AcceptedCmp1: aceitou a campanha 1
- AcceptedCmp2: aceitou a campanha 2
- AcceptedCmp3: aceitou a campanha 3
- AcceptedCmp4: aceitou a campanha 4
- AcceptedCmp5: aceitou a campanha 5
- Complain: reclamações
- Z_CostContact: NA
- Z_Revenue: NA
- Response: 0 ou 1



   <a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">CONTEÚDO DO PROJETO</p>   
    
* [1. IMPORTANDO BIBLIOTECAS](#1)
    
* [2. CARREGANDO DADOS e EDA](#2)
    
* [3. LIMPEZA DOS DADOS](#3)
    
* [4. PRÉ-PROCESSAMENTO DOS DADOS](#4)   
    
* [5. REDUÇÃO DE DIMENSIONALIDADE](#5) 
      
* [6. CLUSTERIZAÇÃO](#6)
    
* [7. AVALIAÇÃO DOS MODELOS](#7)
    
* [8. ANÁLISE DOS RESULTADOS E TOMADA DE DECISÃO](#8)
    
* [9. CONCLUSÃO](#9)
    
* [10. FIM](#10)


<a id="1"></a>
# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:120%;text-align:center;border-radius:10px 10px;">1. Importando Bibliotecas</p>

In [ ]:
# Importando as bibliotecas de manipulação de dados, machine learning e visualização de dados

import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
import sklearn.metrics._classification
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)

<a id="2"></a>
# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:120%;text-align:center;border-radius:10px 10px;">2. Carregando o Dataset e EDA</p>

In [ ]:
# Carregando o dataset
data = pd.read_csv("campanha_marketing.csv", sep="\t")
print("Número de registros no dataset:", len(data))
data.head()

In [ ]:
print("Dimensão do dataset:", data.shape)

In [ ]:
# vamos agora fazer nosso famoso EDA automatizado :)
from dataprep.eda import plot, create_report

# Utilizando o método plot para construção do relatório EDA de forma automática dos nomes dos restaurantes
plot(data)

In [ ]:
create_report(data)


<a id="3"></a>
# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:120%;text-align:center;border-radius:10px 10px;">3. Limpeza dos Dados</p>


**Nesta seção, fazemos as seguintes operações:** 
* Limpeza de dados
* Feature Engineering 
* Entendimento das variáveis categóricas

Para obter uma compreensão completa de quais etapas devemos realizar para a tratativa, vamos dar uma olhada mais detalhada dos dados.

**Do resultado da análise exploratória acima, podemos concluir que:**

* Há valores ausentes apenas no salário (mais precisamente 24 registros)
* Dt_Customer que indica a data em que o cliente entrou na base de dados não está como tipo DateTime (está como object)
* Há algumas variáveis categóricas no nosso dataframe (mais precisamente 14), que estão com o tipo "object". Assim, necessitamos fazer o "encoder" e transformá-las em variáveis numéricas. 

Em primeiro lugar, vamos tratar os valores ausentes. Podemos realizar várias operações, tais como:
- Excluir os valoress ausentes
- Preencher com a média (variáveis numéricas) 
- Preencher com a moda (variáveis categóricas)

In [ ]:
# Neste projeto, vamos remover os registros nulos, porque a quantidade de registros é bem baixa e não impacta na performance do modelo
data = data.dropna()
print("O total de registros após a exclusão dos valores ausentes é", len(data))

Nesse próximo passo, vamos verificar por quanto tempo os clientes estão registrados na empresa. Além disso, aplicaremos o tipo DateTime a coluna "Dt_Customer".

In [ ]:
data["Dt_Customer"] = pd.to_datetime(data["Dt_Customer"])
dates = []
for i in data["Dt_Customer"]:
    i = i.date()
    dates.append(i)  
    
# Datas dos registros mais recente e mais antigo
print("O cliente mais novo da base é de:",max(dates))
print("O cliente mais antigo da base é de:",min(dates))


Vamos agora criar uma variável chamada **("Customer_For")**, que indica a quantidade de dias em que o cliente fez sua primeira compra até a data da última venda da loja.

In [ ]:
# Criando uma nova variável "Customer_For"
days = []
d1 = max(dates) #data da última venda da empresa
for i in dates:
    delta = d1 - i
    days.append(delta)
data["Customer_For"] = days
data["Customer_For"] = pd.to_numeric(data["Customer_For"], errors="coerce")

**Ainda realizando a tratativa das variáveis, realizaremos as seguintes operações:**

* Calcular a **"Idade"** do cliente através do campo **"Year_Birth"**
* Criar uma nova variável **"Gastos"**  indicando o total gasto pelo cliente no supermercado durante os 2 anos.
* Criar uma nova variável **"Living_With"** através do campo **"Marital_Status"** para obter a indicação dos casais.
* Criar uma nova variável **"Children"** para indicar o total de crianças na residência.
* Para obter mais dados da família, criar a variável **"Family_Size"**
* Criar uma variável **"Is_Parent"** para indicar se tem filhos
* Por fim, criaremos três categorias na **"Educação"** simplificando sua contagem de valores.
* Remover algumas variáveis redundantes

In [ ]:
# Engenharia de atributos

#Idade do cliente no dia de hoje 
data["Age"] = 2022-data["Year_Birth"]

#Total gasto em todas as compras de produtos
data["Spent"] = data["MntWines"]+ data["MntFruits"]+ data["MntMeatProducts"]+ data["MntFishProducts"]+ data["MntSweetProducts"]+ data["MntGoldProds"]

#Novo campo calculado a partir da situação do estado civil 
data["Living_With"]=data["Marital_Status"].replace({"Married":"Partner", "Together":"Partner", "Absurd":"Alone", "Widow":"Alone", "YOLO":"Alone", "Divorced":"Alone", "Single":"Alone",})

#Variável indicando o total de filhos na família
data["Children"]=data["Kidhome"]+data["Teenhome"]

#Variável com o total de membros da família
data["Family_Size"] = data["Living_With"].replace({"Alone": 1, "Partner":2})+ data["Children"]

#Variável relativa à paternidade
data["Is_Parent"] = np.where(data.Children> 0, 1, 0)

#Segmentação da escolaridade em 3 níveis
data["Education"]=data["Education"].replace({"Basic":"Undergraduate","2n Cycle":"Undergraduate", "Graduation":"Graduate", "Master":"Postgraduate", "PhD":"Postgraduate"})

#Para melhor entendimento
data=data.rename(columns={"MntWines": "Wines","MntFruits":"Fruits","MntMeatProducts":"Meat","MntFishProducts":"Fish","MntSweetProducts":"Sweets","MntGoldProds":"Gold"})

#Eliminando as features redundantes
to_drop = ["Marital_Status", "Dt_Customer", "Z_CostContact", "Z_Revenue", "Year_Birth", "ID"]
data = data.drop(to_drop, axis=1)

Agora que temos novas variáveis, vamos dar uma olhada novamente nas informações gerais

In [ ]:
data.describe()

Os valores acima mostram uma discrepância grande entre o salário mais baixo (1730 dólares) e o salário mais alto (666.666  dólares).

Note que a idade máxima é de 129 anos, conforme nosso cálculo do ano atual menos o ano de nascimento.
Vamos plotar algumas variáveis para ver o comportamento como um todo.

Há, portanto, alguns outliers nas variáveis, como idade e salário. Portanto, vamos retirá-las. Há diversas formas de tratamento de outliers (medidas estatísticas, etc). Aqui o caso é bem simples e vamos apenas fazer um filtro no dataset.

In [ ]:
# Eliminando os valores discrepantes, definindo um limite para idade (90 anos) e renda (menor que 600k).
print("O total de registros antes da exclusão dos outliers é:", len(data))
data = data[(data["Age"]<90)]
data = data[(data["Income"]<600000)]
print("O total de registros após a exclusão dos outliers é:", len(data))

Vamos agora dar uma olhada na correlação entre as variáveis, sem considerar as variáveis categóricas.

In [ ]:
#correlation matrix
corrmat= data.corr()
plt.figure(figsize=(20,20))  
metade = np.triu(data.corr())
sns.heatmap(corrmat,annot=True, cmap='coolwarm', center=0, mask=metade)


<a id="4"></a>
# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:120%;text-align:center;border-radius:10px 10px;">4. Pré-processamento dos Dados</p>

Nessa seção vamos pré-processar os dados para prepará-los para a clusterização

**Realizaremos as seguintes operações na etapa de pré-processamento dos dados:**

* "LabelEncoder" das variáveis categóricas: para transformar nomes em números. Ex: (diretor: 0, analista: 1, gerente: 2, programador: 3 e fundador: 4) 
* Escala das variáveis com "Standard scaler". Exemplo: renda e idade estão em uma escala bem diferente e isto pode causar um grande problema no treino do nosso modelo uma vez o salário por possuir uma escala muito maior que a idade terá uma influência consequentemente muito maior no resultado. 
* Criar um dataframe separado para redução de dimensionalidade

Link interessante sobre encoder, scaling, etc: https://caiquecoelho.medium.com/um-guia-completo-para-o-pr%C3%A9-processamento-de-dados-em-machine-learning-f860fbadabe1

#### Tratamento das variáveis categóricas

In [ ]:
# Obter lista das variáveis categóricas
s = (data.dtypes == 'object')
object_cols = list(s[s].index)

print("Variáveis categóricas no dataset", object_cols)

In [ ]:
# Label encoding para variáveis categóricas
LE=LabelEncoder()
for i in object_cols:
    data[i]=data[[i]].apply(LE.fit_transform)
    
print("Todas as variáveis agora são numéricas")

In [ ]:
# Verificando as duas colunas transformadas
data

#### Escala das variáveis

Muitos algoritmos de aprendizado de máquina têm melhor desempenho quando as variáveis ​​de entrada numéricas são dimensionadas para um intervalo padrão. 

As duas técnicas mais populares para dimensionar dados numéricos antes da modelagem são normalização e padronização. Normalização dimensiona cada variável de entrada separadamente para o intervalo 0-1, que é o intervalo para valores de ponto flutuante onde temos mais precisão. Padronização dimensiona cada variável de entrada separadamente subtraindo a média (chamada centralização) e dividindo pelo desvio padrão para deslocar a distribuição para ter uma média de zero e um desvio padrão de um.

In [ ]:
#Criando uma cópia dos dados
ds = data.copy()

# criando um subconjunto de dataframe, descartando as variáveis "deals accepted" e "promotions"
cols_del = ['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1','AcceptedCmp2', 'Complain', 'Response']
ds = ds.drop(cols_del, axis=1)

# Scaling
scaler = StandardScaler() #valor = (valor — média) / desvioPadão
scaler.fit(ds)
scaled_ds = pd.DataFrame(scaler.transform(ds),columns= ds.columns )
print("As variáveis agora estão na mesma escala.")

In [ ]:
# Dados dimensionados a serem usados para a etapa de redução de dimensionalidade
print("Dataframe a ser usado para modelagem:")
scaled_ds.head()

<a id="5"></a>
# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:120%;text-align:center;border-radius:10px 10px;">5. Redução de Dimensionalidade</p>
Neste problema, existem muitos fatores /características com base nos quais a classificação final será feita. Esses fatores são basicamente atributos ou variáveis. Quanto maior o número de recursos, mais difícil é de se trabalhar com o problema. 

Muitas dessas variáveis estão correlacionadas e, portanto, são redundantes. É por isso que é recomendada a redução de dimensionalidade nos atributos selecionados antes de colocá-los em um estimador. Assim, *redução de dimensionalidade é o processo de redução do número de variáveis aleatórias em consideração, obtendo um conjunto de variáveis principais.*

**Principal component analysis (PCA)** é uma das técnicas para a redução de dimensionalidade de datasets, incrementando a interpretabilidade dos dados mas ao mesmo tempo minimizando a perda de informação. Adicionalmente, o PCA nada mais é que uma técnica a qual transforma atributos com uma certa correlação em um único atributo. O PCA deve ser aplicado apenas em casos em que o seu dataset possui muitas colunase o treino do seu modelo acaba por ser muito demorado ou inviável devido ao alto número de colunas.

**Operações a serem realizadas nessa etapa:**
* Redução de dimensionalidade com PCA
* Plotar o dataset reduzido após a seleção das variáveis

**Redução de dimensionalidade com PCA**

Para esse projeto, reduziremos para 3 dimensões, assim facilitando a visualizacão da informação.

In [ ]:
#Inicializando o PCA com 3 componentes
pca = PCA(n_components=3)
pca.fit(scaled_ds)
PCA_ds = pd.DataFrame(pca.transform(scaled_ds), columns=(["col1","col2","col3"]))
PCA_ds.describe()

In [ ]:
PCA_ds.info()

In [ ]:
# Uma projeção 3D de dados na dimensão reduzida
x =PCA_ds["col1"]
y =PCA_ds["col2"]
z =PCA_ds["col3"]
#To plot
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(x,y,z, c="blue", marker="o" )
ax.set_title("Uma projeção 3D dos dados em uma dimensão reduzida")
plt.show()

<a id="6"></a>
# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:120%;text-align:center;border-radius:10px 10px;">6. Clusterização</p>

Agora que temos a dimensão reduzida para 3 atributos, iremos realizar a clusterização utilizando a técnica de cluster aglomerativo. "Agglomerative Clustering" é um método de clusterização hierárquica. Envolve exemplos de agrupamento até o número de clusters apropriado ser encontrado. 

Algoritmos desse tipo têm como objetivo agrupar dados similares entre si usando ferramentas como a distância entre eles. A diferença desse tipo de algoritmo para os demais está na criação de diversos clusters (alguns agrupamentos dentro de outros), o que, por fim, acaba gerando uma árvore de clusters, na qual um dado pertence a grupos menores e maiores, criando, dessa forma, uma hierarquia.

Contudo, esses algoritmos apresentam duas desvantagens principais:

Não lidam muito bem com outliers, tendo em vista que, graças a lógica do algoritmo, tendem a classificar o dado distante como um novo cluster, ou, em piores casos, acabam deformando outro cluster para incluí-lo.
Não costumam ser utilizados em larga escala e em quantidades grandes de dados devido a sua complexidade e ineficiência com muitos dados. Por isso, estuda-se esses métodos uma vez que servem de base para soluções mais sofisticadas (como o density-based).

**Passos envolvidos na clusterização:**
* Método Elbow para determinar o número de clusters a serem formados
* Clusterização utilizando o "Agglomerative Clustering"
* Examinar os clusters através de visualização gráfica

**O Método Elbow (Cotovelo)**
A ideia é rodar o KMeans para vários quantidades diferentes de clusters e dizer qual dessas quantidades é o número ótimo de clusters. O que geralmente acontece ao aumentar a quantidade de clusters no KMeans é que as diferenças entre clusters se tornam muito pequenas, e as diferenças das observações intra-clusters vão aumentando. Então é preciso achar um equilíbrio em que as observações que formam cada agrupamento sejam o mais homogêneas possível e que os agrupamentos formados sejam o mais diferentes um dos outros.  
Mais info: https://medium.com/pizzadedados/kmeans-e-metodo-do-cotovelo-94ded9fdf3a9

In [ ]:
# Exame rápido do "método do cotovelo" para encontrar o número de clusters a serem feitos.
print('Método Elbow Methodpara determinar o número ótimo de clusters.')
k= range(2,10)
Elbow_M = KElbowVisualizer(KMeans(), k=k, random_state=42)
Elbow_M.fit(PCA_ds)
Elbow_M.show()

O gráfico acima indica que 4 é o número ótimo de clusters para os nossos dados.
A seguir, vamos fazer o fit de Agglomerative Clustering para obter os clusters finais. 

#### Parâmetros Iniciais de Entrada do Agglomerative Clustering

**n_clusters**: número de clusters que queremos obter/acreditamos que existe  
**affinity**: forma de medir a distância (afinidade) entre os datapoints  
**connectivity**: matriz de conectividade que define os vizinhos para cada ponto  
**compute_full_tree**: parâmetro que permite a “parada” do algoritmo caso o dendograma já apresente o número de clusters previsto em n_clusters. É útil para reduzir o tempo de computação dos Clusters.  
**linkage**: critério de ligação escolhido  
**distance_threshold**: limite o qual permite que os pontos possam ser agrupados em um cluster. Se essa distância for maior, o algoritmo não os junta.

#### Funcionamento Agglomerative Clustering

O algoritmo se dá da seguinte forma:

- Ache dois pontos que estejam o mais próximo possível
- Substitua eles por um ponto com a média dos valores deles
- Repita o processo até que todos tenham um único cluster
- A proximidade dos pontos é definida pelo cálculo da sua distância, sendo o método mais comum para isso a distância euclidiana.

In [ ]:
#Inicilizando o modelo Agglomerative Clustering  
AC = AgglomerativeClustering(n_clusters=4)

# Ajustar o modelo e prever os clusters
clusters_formados = AC.fit_predict(PCA_ds)
PCA_ds["Clusters"] = clusters_formados

# Adicionando a variável Clusters ao dataframe original.
data["Clusters"]= clusters_formados

Para examinar os clusters formados, vamos dar uma olhada na distribuição 3-D dos clusters.

In [ ]:
# Plotando os clusters
fig = plt.figure(figsize=(10,8))
ax = plt.subplot(111, projection='3d')
ax.scatter(x, y, z, s=40, c=PCA_ds["Clusters"], marker='o', cmap ='coolwarm' )
ax.set_title("Visualizando os clusters")
plt.show()

<a id="7"></a>
# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:120%;text-align:center;border-radius:10px 10px;">7. Avaliação do Modelo</p>

Uma vez que este é um algoritmo não supervisionado, não temos uma variável target marcada para avaliar ou pontuar o modelo. O objetivo desta seção é estudar os padrões nos clusters formados e determinar a natureza dos padrões dos clusters.

Para isso, analisaremos os dados dos clusters por meio de análise exploratória de dados e assim tirar algumas conclusões.

**Primeiramente, vamos ver a distribuição do grupo de clusters**

In [ ]:
#Plotando os registros de cada cluster
pal = 'coolwarm'
pl = sns.countplot(x=data["Clusters"], palette= pal)
pl.set_title("Distribuição dos clusters")
plt.show()

Os clusters parecem estar distribuídos de forma bastante uniforme.

In [ ]:
pl = sns.scatterplot(data = data,x=data["Spent"], y=data["Income"],hue=data["Clusters"], palette= pal)
pl.set_title("Clusters baseados no Salário e no Gasto")
plt.legend()
plt.show()

**Salário vs Gastos mostra pra gente os seguintes padrões**
* Grupo 0: Alto Gasto & Médio Salário
* Grupo 1: Alto Gasto & Alto Salário
* Grupo 2: Baixo Gasto & Baixo Salário
* Grupo 3: Baixo Gasto & Médio Salário  



In [ ]:
plt.figure()

pl=sns.boxenplot(x=data["Clusters"], y=data["Spent"], palette=pal)
plt.show()

Agora vamos explorar uma parte muito importante do projeto, que é o que cada cluster está gastando nas estratégias de marketing direcionadas.

In [ ]:
#Criando uma variável que é a soma de todas as promoções aceitas
data["Total_Promos"] = data["AcceptedCmp1"]+ data["AcceptedCmp2"]+ data["AcceptedCmp3"]+ data["AcceptedCmp4"]+ data["AcceptedCmp5"]

#Plotando os dados das campanhas
plt.figure()
pl = sns.countplot(x=data["Total_Promos"],hue=data["Clusters"], palette= pal)
pl.set_title("Promoções aceitas")
pl.set_xlabel("Número total de promoções aceitas")
plt.show()

As campanhas de marketing não estão surtindo o efeito desejsado. Muito poucos participantes no geral. Além disso, nenhum cluster participou de todas as 5 campanhas. Talvez sejam necessárias campanhas mais bem direcionadas e bem planejadas para aumentar as vendas.


In [ ]:
#Plotando o número de ofertas compradas
plt.figure()
pl=sns.boxenplot(y=data["NumDealsPurchases"],x=data["Clusters"], palette= pal)
pl.set_title("Número de ofertas compradas")
plt.show()

Ao contrário das campanhas, as ofertas oferecidas tiveram boa conversão. Tem melhor resultado o cluster 0 e o cluster 3.
Nada parece atrair o cluster 1 nem o cluster 2.


In [ ]:
# Verificar os detalhes das compras realizadas por canais
Canais =["NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases",  "NumWebVisitsMonth"] 

for i in Canais:
    plt.figure()
    sns.jointplot(x=data[i],y = data["Spent"],hue=data["Clusters"], palette= pal)
    plt.show()

<a id="8"></a>
# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:120%;text-align:center;border-radius:10px 10px;">8. Análise dos Resultados e Tomada de Decisão</p>

Agora que formamos os clusters e analisamos seus hábitos de compra, vamos ver quem são todos esses clusters. Para isso, vamos traçar o perfil dos clusters formados e chegar a uma conclusão sobre quem é nosso cliente estrela e quem precisa de mais atenção da equipe de marketing da loja.

Para decidir sobre, vamos plotar alguns gráficos que são indicativos das características pessoais e familiares do cliente à luz do cluster em que ele está. Com base nos resultados, faremos nossas conclusões.

In [ ]:
Personal = [ "Kidhome","Teenhome", "Age", "Children", "Family_Size", "Is_Parent", "Education","Living_With"]

for i in Personal:
    plt.figure()
    sns.jointplot(x=data[i], y=data["Spent"], hue =data["Clusters"], kind="kde", palette=pal)
    plt.show()


**Análise e Tomada de Decisão:**

As informações a seguir podem ser deduzidas sobre os clientes nos diferentes clusters.

### CLUSTER 0
- São definitivamente pais
- Tem pelo menos 2 membros na família e no máximo 4
- Pais sem filhos são um sub-grupo deste grupo
- A maioria tem um adolescente em casa
- São relativamente mais velhos (em idade)

### CLUSTER 1
- Definitivamente não são pais
- Máximo de 2 pessoas por família
- Há uma quantidade considerável de pessoas solteiras
- São de todas as idades
- Grupo que possui um salário alto

### CLUSTER 2
- A maioria das pessoas são pais
- O máximo de membros na família é 3
- A maioria tem uma criança (e não adolescentes)
- São relativamente mais jovens

### CLUSTER 3
- São definitivamente pais
- Tem pelo menos 2 membros na família e no máximo 5
- A maioria tem um adolescente em casa
- São relativamente mais velhos (em idade)
- Grupo com baixa-média renda


  

<a id="9"></a>
# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:120%;text-align:center;border-radius:10px 10px;">9. Conclusão</p>


Neste projeto, realizamos um agrupamento não supervisionado. Fizemos várias operações incluindo redução de dimensionalidade seguida de agrupamento aglomerativo. Criamos 4 clusters e os usamos ainda mais na criação de perfis de clientes em clusters de acordo com suas estruturas familiares e renda/gastos.
Isso pode ser usado no planejamento de melhores estratégias de marketing.

Como exemplo, caso queiram ofertar um novo brinquedo para crianças, realizar a divulgação somente para os clientes do cluster 2, para ter maior assertividade.
Dessa maneira, há inúmeras formas de utilizar os clusters obtidos para realizar campanhas de marketing mais inteligentes. 

<a id="10"></a>
# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:120%;text-align:center;border-radius:10px 10px;">10 - Fim</p>